## First we need all necessary imports

In [1]:
import os
import numpy as np
import flopy.modflow as mf
import shutil
import geojson
import pyproj

* The modflow-files will be stored in './workspace' 
* Each calculation will delete the files

In [2]:
import json
f = open('configuration.json')
configuration = json.load(f)
f.close()
modflow_data = configuration['data']['mf']

platform_model_workspace = os.path.join('platform_model')

if os.path.exists(platform_model_workspace):
    shutil.rmtree(platform_model_workspace)
    
if not os.path.exists(platform_model_workspace):
    os.makedirs(platform_model_workspace)

import MfPackages
mf_data_obj = modflow_data['mf']
mf_data_obj['modelname'] = 'lak_test'
mf_data_obj['exe_name'] = '../modflow/mf2005'
mf_data_obj['model_ws'] = platform_model_workspace
platform_model = MfPackages.MfAdapter(mf_data_obj).get_package()
MfPackages.DisAdapter(modflow_data['dis']).get_package(platform_model)
MfPackages.BasAdapter(modflow_data['bas']).get_package(platform_model)
MfPackages.ChdAdapter(modflow_data['chd']).get_package(platform_model)
MfPackages.LakAdapter(modflow_data['lak']).get_package(platform_model)
MfPackages.LpfAdapter(modflow_data['lpf']).get_package(platform_model)
MfPackages.PcgAdapter(modflow_data['pcg']).get_package(platform_model)
MfPackages.OcAdapter(modflow_data['oc']).get_package(platform_model)


platform_model.write_input()
platform_model.run_model()


FloPy is using the following executable to run the model: ../../modflow/mf2005

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.12.00 2/3/2017                        

 Using NAME file: lak_test.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2023/08/24 11:13:03

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2023/08/24 11:13:03
 Elapsed run time:  0.002 Seconds

  Normal termination of simulation


(True, [])

In [3]:
flopy_model_workspace = os.path.join('flopy_model')

if os.path.exists(flopy_model_workspace):
    shutil.rmtree(flopy_model_workspace)
    
if not os.path.exists(flopy_model_workspace):
    os.makedirs(flopy_model_workspace)
    
flopy_model = mf.Modflow(
    modelname='lak_test', 
    exe_name='../modflow/mf2005', 
    model_ws=flopy_model_workspace
)

nlay = 1
nrow = 6
ncol = 11
delr = [37871., 37872., 37871., 37871., 37871., 37867., 37871., 37871. , 37871., 37871., 37871.]
delc = [57612., 57609., 57612., 57612., 57609., 57612.]

mf.ModflowDis(flopy_model, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc, top=460, botm=390, nper=1, steady=True)

spd = {
    0: [
        [0, 0, 0, 450, 450],
        [0, 1, 0, 450, 450],
        [0, 2, 0, 450, 450],
        [0, 3, 0, 450, 450],
        [0, 4, 0, 450, 450],
        [0, 5, 0, 450, 450],
        [0, 0, 10, 445, 445],
        [0, 1, 10, 445, 445],
        [0, 2, 10, 445, 445],
        [0, 3, 10, 445, 445],
        [0, 4, 10, 445, 445],
        [0, 5, 10, 445, 445],
    ]
}

mf.ModflowChd(flopy_model, stress_period_data=spd)

lakArr = np.zeros((nlay,nrow,ncol))
lakArr[0, 2, 4] = 1
lakArr[0, 2, 5] = 1
lakArr[0, 2, 6] = 1
lakArr[0, 3, 4] = 1
lakArr[0, 3, 5] = 1
lakArr[0, 3, 6] = 1
ibound = np.ones((nlay,nrow,ncol))
wetdry = -0.01 * np.ones((nlay,nrow,ncol))
ibound = np.where(lakArr > 0, 0, ibound) #IBOUND should be set to zero for every lake cell (LKARR >0)
wetdry = np.where(lakArr > 0, 0, wetdry) #WETDRY should be set to zero for every lake cell (LKARR >0)
bdlknc=np.zeros((nlay,nrow,ncol))
bdlknc = np.where(lakArr ==1, 10.0, bdlknc) #setting bed leakance value for lake 1
bdlknc = np.where(lakArr ==2, 20.0, bdlknc) #setting bed leakance value for lake 2
flux_data={0:[[0.003, 0.0073, 0.0, 0.0],
              [0.005, 0.0073, 0.0, 0.0]]}

mf.ModflowLak(
    flopy_model,
    nlakes=1,
    theta=-1,
    nssitr=0,
    sscncr=0.001,
    surfdep=0,
    stages=[449.0],
    stage_range=[[445.0,450.0]],
    lakarr=lakArr,
    bdlknc=bdlknc,
    flux_data=flux_data,
    unit_number=16,
    lwrt=0,
)

mf.ModflowBas(flopy_model, ibound=ibound, strt=455.0)
mf.ModflowLpf(flopy_model, hk=8.64, vka=0.864, ss=1e-5, sy=0.2, wetdry=wetdry)
mf.ModflowPcg(flopy_model, rclose=1e-2, hclose=1e-2)
mf.ModflowOc(flopy_model)

flopy_model.write_input()
flopy_model.run_model()

FloPy is using the following executable to run the model: ../../modflow/mf2005

                                  MODFLOW-2005     
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL
                             Version 1.12.00 2/3/2017                        

 Using NAME file: lak_test.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2023/08/24 11:13:03

 Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.
 Run end date and time (yyyy/mm/dd hh:mm:ss): 2023/08/24 11:13:03
 Elapsed run time:  0.002 Seconds

  Normal termination of simulation


(True, [])

### Read calculated data

In [4]:
import flopy.utils as fu

hds = fu.HeadFile(os.path.join(platform_model_workspace, 'lak_test.hds'))
h = hds.get_data(kstpkper=(0,0))
h

array([[[450.     , 449.55933, 449.10083, 448.60574, 448.06143,
         447.5    , 446.93854, 446.39426, 445.89917, 445.44067,
         445.     ],
        [450.     , 449.6005 , 449.18555, 448.71964, 448.1012 ,
         447.5    , 446.89877, 446.28036, 445.81445, 445.3995 ,
         445.     ],
        [450.     , 449.67746, 449.38818, 449.18646, 449.     ,
         449.     , 449.     , 445.81354, 445.61182, 445.32254,
         445.     ],
        [450.     , 449.67746, 449.38818, 449.18646, 449.     ,
         449.     , 449.     , 445.81354, 445.61182, 445.32254,
         445.     ],
        [450.     , 449.6005 , 449.18555, 448.71964, 448.1012 ,
         447.5    , 446.89877, 446.28036, 445.81445, 445.3995 ,
         445.     ],
        [450.     , 449.55933, 449.10083, 448.60574, 448.06143,
         447.5    , 446.93854, 446.39426, 445.89917, 445.44067,
         445.     ]]], dtype=float32)

In [5]:
import flopy.utils as fu

hds = fu.HeadFile(os.path.join(flopy_model_workspace, 'lak_test.hds'))
h = hds.get_data(kstpkper=(0,0))
h

array([[[450.     , 449.55933, 449.10083, 448.60574, 448.06143,
         447.5    , 446.93854, 446.39426, 445.89917, 445.44067,
         445.     ],
        [450.     , 449.6005 , 449.18555, 448.71964, 448.1012 ,
         447.5    , 446.89877, 446.28036, 445.81445, 445.3995 ,
         445.     ],
        [450.     , 449.67746, 449.38818, 449.18646, 449.     ,
         449.     , 449.     , 445.81354, 445.61182, 445.32254,
         445.     ],
        [450.     , 449.67746, 449.38818, 449.18646, 449.     ,
         449.     , 449.     , 445.81354, 445.61182, 445.32254,
         445.     ],
        [450.     , 449.6005 , 449.18555, 448.71964, 448.1012 ,
         447.5    , 446.89877, 446.28036, 445.81445, 445.3995 ,
         445.     ],
        [450.     , 449.55933, 449.10083, 448.60574, 448.06143,
         447.5    , 446.93854, 446.39426, 445.89917, 445.44067,
         445.     ]]], dtype=float32)

In [6]:
flopy_bas = flopy_model.get_package('BAS6')
platform_bas = platform_model.get_package('BAS6')


if (flopy_bas.strt.array == platform_bas.strt.array).all():
    print('strt is equal')
else:
    print(flopy_bas.strt.array)
    print(platform_bas.strt.array)
    
if (flopy_bas.ibound.array == platform_bas.ibound.array).all():
    print('ibound is equal')
else:
    print(flopy_bas.ibound.array)
    print(platform_bas.ibound.array)
    
if flopy_bas.heading == platform_bas.heading:
    print('heading is equal')
else:
    print('heading is not equal')
    
if flopy_bas.options == platform_bas.options:
    print('options is equal')
else:
    print('options is not equal')
    
if flopy_bas.ixsec == platform_bas.ixsec:
    print('ixsec is equal')
else:
    print('ixsec is not equal')
    
if flopy_bas.ichflg == platform_bas.ichflg:
    print('ichflg is equal')
else:
    print('ichflg is not equal')
    
if flopy_bas.stoper == platform_bas.stoper:
    print('stoper is equal')
else:
    print('stoper is not equal')
    
if flopy_bas.hnoflo == platform_bas.hnoflo:
    print('hnoflo is equal')
else:
    print('hnoflo is not equal')
    

strt is equal
ibound is equal
heading is equal
options is equal
ixsec is equal
ichflg is equal
stoper is equal
hnoflo is equal


In [7]:
flopy_dis = flopy_model.get_package('DIS')
platform_dis = platform_model.get_package('DIS')

if flopy_dis.nrow == platform_dis.nrow:
    print('nrow is equal')
else:
    print('nrow is not equal')
if flopy_dis.ncol == platform_dis.ncol:
    print('ncol is equal')
else:
    print('ncol is not equal')
if flopy_dis.nlay == platform_dis.nlay:
    print('nlay is equal')
else:
    print('nlay is not equal')
if flopy_dis.nper == platform_dis.nper:
    print('nper is equal')
else:
    print('nper is not equal')
if (flopy_dis.laycbd.array == platform_dis.laycbd.array).all():
    print('laycbd is equal')
else:
    print('laycbd is not equal')
    print(flopy_dis.laycbd.array)
    print(platform_dis.laycbd.array)
if (flopy_dis.delr.array == platform_dis.delr.array).all():
    print('delr is equal')
else:
    print('delr is not equal')
    print(flopy_dis.delr.array)
    print(platform_dis.delr.array)    
    
if (flopy_dis.delc.array == platform_dis.delc.array).all():
    print('delc is equal')
else:
    print('delc is not equal')
    print(flopy_dis.delc.array)
    print(platform_dis.delc.array)    
    
if (flopy_dis.top.array == platform_dis.top.array).all():
    print('top is equal')
else:
    print('top is not equal')
    print(flopy_dis.top.array)
    print(platform_dis.top.array)
    
if (flopy_dis.botm.array == platform_dis.botm.array).all():
    print('botm is equal')
else:
    print('botm is not equal')
    print(flopy_dis.botm.array)
    print(platform_dis.botm.array)
    
if (flopy_dis.perlen.array == platform_dis.perlen.array).all():
    print('perlen is equal')
else:
    print('perlen is not equal')
    print(flopy_dis.perlen.array)
    print(platform_dis.perlen.array)
    
if (flopy_dis.nstp.array == platform_dis.nstp.array).all():
    print('nstp is equal')
else:
    print('nstp is not equal')
    print(flopy_dis.nstp.array)
    print(platform_dis.nstp.array)
    
if (flopy_dis.tsmult.array == platform_dis.tsmult.array).all():
    print('tsmult is equal')
else:
    print('tsmult is not equal')
    print(flopy_dis.tsmult.array)
    print(platform_dis.tsmult.array)
    
if (flopy_dis.steady.array == platform_dis.steady.array).all():
    print('steady is equal')
else:
    print('steady is not equal')
    print(flopy_dis.steady.array)
    print(platform_dis.steady.array)
    
if flopy_dis.itmuni == platform_dis.itmuni:
    print('itmuni is equal')
else:
    print('itmuni is not equal')    
    
if flopy_dis.lenuni == platform_dis.lenuni:
    print('lenuni is equal')
else:
    print('lenuni is not equal')

nrow is equal
ncol is equal
nlay is equal
nper is equal
laycbd is equal
delr is equal
delc is equal
top is equal
botm is equal
perlen is equal
nstp is equal
tsmult is equal
steady is equal
itmuni is equal
lenuni is equal


In [8]:
flopy_chd = flopy_model.get_package('CHD')
platform_chd = platform_model.get_package('CHD')

if (flopy_chd.stress_period_data.array['shead'] == platform_chd.stress_period_data.array['shead']).all():
    print('shead is equal')
else:
    print('shead is not equal')
    print(flopy_chd.stress_period_data.array['shead'])
    print(platform_chd.stress_period_data.array['shead'])
    
if (flopy_chd.stress_period_data.array['ehead'] == platform_chd.stress_period_data.array['ehead']).all():
    print('ehead is equal')
else:
    print('ehead is not equal')
    print(flopy_chd.stress_period_data.array['ehead'])
    print(platform_chd.stress_period_data.array['ehead'])


shead is not equal
[[[[450.  nan  nan  nan  nan  nan  nan  nan  nan  nan 445.]
   [450.  nan  nan  nan  nan  nan  nan  nan  nan  nan 445.]
   [450.  nan  nan  nan  nan  nan  nan  nan  nan  nan 445.]
   [450.  nan  nan  nan  nan  nan  nan  nan  nan  nan 445.]
   [450.  nan  nan  nan  nan  nan  nan  nan  nan  nan 445.]
   [450.  nan  nan  nan  nan  nan  nan  nan  nan  nan 445.]]]]
[[[[450.  nan  nan  nan  nan  nan  nan  nan  nan  nan 445.]
   [450.  nan  nan  nan  nan  nan  nan  nan  nan  nan 445.]
   [450.  nan  nan  nan  nan  nan  nan  nan  nan  nan 445.]
   [450.  nan  nan  nan  nan  nan  nan  nan  nan  nan 445.]
   [450.  nan  nan  nan  nan  nan  nan  nan  nan  nan 445.]
   [450.  nan  nan  nan  nan  nan  nan  nan  nan  nan 445.]]]]
ehead is not equal
[[[[450.  nan  nan  nan  nan  nan  nan  nan  nan  nan 445.]
   [450.  nan  nan  nan  nan  nan  nan  nan  nan  nan 445.]
   [450.  nan  nan  nan  nan  nan  nan  nan  nan  nan 445.]
   [450.  nan  nan  nan  nan  nan  nan  nan  nan  nan 44

In [9]:

flopy_lak = flopy_model.get_package('LAK')
platform_lak = platform_model.get_package('LAK')

if flopy_lak.nlakes == platform_lak.nlakes:
    print('nlakes is equal')
else:
    print('nlakes is not equal')
    
if flopy_lak.theta == platform_lak.theta:
    print('theta is equal')
else:
    print('theta is not equal')
    
if flopy_lak.nssitr == platform_lak.nssitr:
    print('nssitr is equal')
else:
    print('nssitr is not equal')
    
if flopy_lak.sscncr == platform_lak.sscncr:
    print('sscncr is equal')
else:
    print('sscncr is not equal')

if flopy_lak.surfdep == platform_lak.surfdep:
    print('surfdep is equal')
else:
    print('surfdep is not equal')
    print(flopy_lak.surfdep)
    print(platform_lak.surfdep)
    
if (flopy_lak.stages == platform_lak.stages).all():
    print('stages is equal')
else:
    print('stages is not equal')
    print(flopy_lak.stages)
    print(platform_lak.stages)
    
if (flopy_lak.stage_range == platform_lak.stage_range).all():
    print('stage_range is equal')
else:
    print('stage_range is not equal')
    print(flopy_lak.stage_range)
    print(platform_lak.stage_range)
    
if (flopy_lak.lakarr.array == platform_lak.lakarr.array).all():
    print('lakarr is equal')
else:
    print('lakarr is not equal')
    print(flopy_lak.lakarr)
    print(platform_lak.lakarr)
    
if (flopy_lak.bdlknc.array == platform_lak.bdlknc.array).all():
    print('bdlknc is equal')
else:
    print('bdlknc is not equal')
    print(flopy_lak.bdlknc.array)
    print(platform_lak.bdlknc.array)

if flopy_lak.flux_data == platform_lak.flux_data:
    print('flux_data is equal')
else:
    print('flux_data is not equal')
    print(flopy_lak.flux_data)
    print(platform_lak.flux_data)

nlakes is equal
theta is equal
nssitr is equal
sscncr is equal
surfdep is equal
stages is equal
stage_range is equal
lakarr is equal
bdlknc is not equal
[[[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0. 10. 10. 10.  0.  0.  0.  0.]
   [ 0.  0.  0.  0. 10. 10. 10.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]]]
[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0.]
   [0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
   [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]]
flux_data is not equal
{0: [[0.003, 0.0073, 0.0, 0.0], [0.005, 0.0073, 0.0, 0.0]]}
{0: [[0.05, 0.075, 0, 0, 95, 95]], 1: [[0.05, 0.075, 0, 0, 95, 95]], 2: [[0.05, 0.075, 0, 0, 95, 95]]}


In [10]:
flopy_lak.flux_data

{0: [[0.003, 0.0073, 0.0, 0.0], [0.005, 0.0073, 0.0, 0.0]]}